In [2]:
import pandas as pd
import json
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import re
from dateutil import parser
import json
import ipaddress

In [3]:
ignoreList = ['.length ']
matchPatterns = []
def tokenize_filter(filter_dict):
    tokens = []
    if filter_dict is not None:
        filter_str = json.dumps(filter_dict)
        tokens = re.findall(r"\[\w']+|\[.,!?;\]", filter_str)
    return tokens

def replace_alpha_words(text):
    if text is None:
        return("")
    else:
        regex = r"^(?!\$)\w+"
        return re.sub(r"'[\w ]+'", '\'name\'', text)

def replace_this_var(text):
    matched = (re.findall(r"\.[\w]+",text))
    #matchPatterns.append(matched)
    for matches in matched:
        if str(matches) != '.length':
            text = re.sub(matches,".name",text)
        else:
            matchPatterns.append(matches)
    return text

def replace_this_var_condi(text):
    regex = r"^(?!\$)\w+"
    return re.sub(r"\.[\w]+[\.]",".name.",text)

In [4]:
labelData = pd.read_json("../No-SQL_Gen/No-SqlDataset.json")
labelData['text'] = labelData['text'].apply(lambda x: x.replace('"', "'") if isinstance(x, str) else x)
labelData.head()

,text,label
0,"{'user': '{}', 'password': '{}'}",0
1,{'username': {'$regex': '^{}'}},1
2,{'username': {'$ne': '{}'}},1
3,{'username': {'$gt': '{}'}},1
4,"{'user': 'hacker', 'password': {'$ne': ''}}",1


In [5]:
logData = pd.read_json('../Dataset/queryLogs.json')
logData.head()
logData['t'] = logData['t'].apply(lambda x: parser.parse(x['$date']).timestamp())
attr_cols = ['type', 'ns', 'command', 'planSummary', 'planningTimeMicros', 'keysExamined', 'docsExamined', 'nBatches', 'cursorExhausted', 'numYields', 'nreturned', 'queryFramework', 'reslen', 'locks', 'storage', 'cpuNanos', 'remote', 'protocol', 'durationMillis']
for col in attr_cols:
    logData[col] = logData['attr'].apply(lambda x: x.get(col, None))
logData['FindCollectionTarget'] = logData['command'].apply(lambda x: x.get('find', None) if isinstance(x, dict) else "None")
logData['filter'] = logData['command'].apply(lambda x: x.get('filter', None) if isinstance(x, dict) else None)
logData['tokenized_filter'] = logData['filter'].apply(lambda x: tokenize_filter(str(x)) if x is not None else None)
logData['docUUID'] = logData['command'].apply(lambda x: x.get('lsid', {}).get('id', {}).get('$uuid', None) if isinstance(x, dict) else None)
logData['db'] = logData['command'].apply(lambda x: x.get('$db', None) if isinstance(x, dict) else None)
logData['db'] = logData['command'].apply(lambda x: x.get('$db', None) if isinstance(x, dict) else None)
logData['filter_str'] = logData['filter'].apply(lambda x: str(x) if x is not None else None)
logData['returnDataSize'] = logData['command'].apply(lambda x: x.get('nreturned', None) if isinstance(x, dict) else None)
#nreturned
logData['cpuNanos'].head()

0    176371
1    112781
2     71093
3     81173
4    141175
Name: cpuNanos, dtype: int64

In [6]:
labelData = pd.read_json("../No-SQL_Gen/No-SqlDataset.json")
labelData['text'] = labelData['text'].apply(lambda x: x.replace('"', "'") if isinstance(x, str) else x)
labelData.head()
merged_df = pd.merge(logData[['t','filter_str','remote','FindCollectionTarget','db','docUUID', 'nreturned', 'cpuNanos', 'planningTimeMicros']], labelData[['text', 'label']], how='left', left_on='filter_str', right_on='text')
merged_df = merged_df.drop('text',axis=1)
merged_df['FindCollectionTargetdeNamed'] = merged_df['FindCollectionTarget'].apply(replace_alpha_words)
merged_df['filter_str'] = merged_df['filter_str'].apply(lambda x: x.replace('"', "'") if isinstance(x, str) else x)
merged_df['filter_str'] = merged_df['filter_str'].apply(lambda x : str(x))
merged_df['queryLength'] = merged_df['filter_str'].apply(lambda x: len(str(x)))
merged_df['whereDetected'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"\$where", x)) else 0)
merged_df['regexDetected'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"\$regex", x)) else 0)
merged_df['emptyFilter'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"^\{\}$", x)) else 0)
merged_df['nullArgument'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"\{\}'", x)) else 0)
merged_df['logicOperator'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"[=><]+", x)) else 0)
merged_df['denamed'] = merged_df['filter_str'].apply(replace_alpha_words)
merged_df['denamed'] = merged_df['denamed'].apply(replace_this_var)
merged_df['denamedDb'] = merged_df['db'].apply(replace_alpha_words)
merged_df['ipHash'] = merged_df['remote'].apply(lambda ip: hash(int(ipaddress.IPv4Address(ip.split(':')[0]))))
merged_df['port'] = merged_df['remote'].apply(lambda ip: ip.split(':')[1])
merged_df = merged_df.dropna()


In [7]:
final_df = merged_df[['t','filter_str','docUUID','denamed','FindCollectionTarget','FindCollectionTargetdeNamed','queryLength','ipHash','port','whereDetected','regexDetected','emptyFilter','nullArgument','logicOperator','db','denamedDb','nreturned','cpuNanos','label','planningTimeMicros']]
print(final_df.head())
print("Shape of the DataFrame:", final_df.shape)
print("\nData Types:")
print(final_df.dtypes)
print("\nDescriptive Statistics:")
print(final_df.describe())
print("\nNull Values:")
print(final_df.isnull().sum())

final_df.to_csv('../Dataset/final.csv', index=False)

stats = {
    'shape': final_df.shape,
    'dtypes': final_df.dtypes.apply(lambda x: x.name).to_dict(),
    'descriptive_stats': final_df.describe().to_dict(),
    'null_values': final_df.isnull().sum().to_dict()
}

with open('../Dataset/metadata.json', 'w') as file:
    json.dump(stats, file, indent=4)

              t                        filter_str  \
0  1.712243e+09  {'user': '{}', 'password': '{}'}   
1  1.712243e+09  {'user': '{}', 'password': '{}'}   
2  1.712243e+09   {'username': {'$regex': '^{}'}}   
3  1.712243e+09   {'username': {'$regex': '^{}'}}   
4  1.712243e+09       {'username': {'$ne': '{}'}}   

                                docUUID                       denamed  \
0  e5e23d7e-5367-4829-a104-ca369c49d4dc  {'name': '{}', 'name': '{}'}   
1  e5e23d7e-5367-4829-a104-ca369c49d4dc  {'name': '{}', 'name': '{}'}   
2  e5e23d7e-5367-4829-a104-ca369c49d4dc   {'name': {'$regex': '^{}'}}   
3  e5e23d7e-5367-4829-a104-ca369c49d4dc   {'name': {'$regex': '^{}'}}   
4  e5e23d7e-5367-4829-a104-ca369c49d4dc       {'name': {'$ne': '{}'}}   

  FindCollectionTarget FindCollectionTargetdeNamed      ipHash   port  \
0      test_collection             test_collection  2130706433  47038   
1      test_collection             test_collection  2130706433  47038   
2      test_collection 